In [6]:
# import statements to download census data
import requests
import pandas as pd
from google.colab import files
import json

## To download desired dataset first go to : https://api.census.gov/data/2023/acs/acs5/profile/variables.html

### Find the "Name" code for the desired dataset by pressing "ctrl + F" in your keyboard and paste it in user input box after running the code below.

#### Note: The link provided is from 2023 and ACS 5 year estimates, the nomenclature followed by U.S. Census Bureau remains the same throughout i.e., the naming is the same for a particular dataset even if it is ACS 1 or 3 year estimate or even if the year is different.

In [4]:
def download_census_data():
    print("Welcome to Census API Downloader for ACS Profile Data")
    print("This script will construct the API URL based on your inputs and download the data as CSV.")

    # Input year
    year = input("Enter year (2009-2023, default 2023): ") or "2023"

    # Input ACS type
    acs = input("Enter ACS type (acs1, acs3, acs5, default acs5): ") or "acs5"

    # Input variables
    variables_str = input("Enter variable codes separated by commas (e.g., DP04_0058E,DP03_0062E): ")
    variables = [v.strip() for v in variables_str.split(',') if v.strip()]
    get_params = 'NAME' + (',' + ','.join(variables) if variables else '')

    # Input geography level
    level = input("Enter geography level (state, county, tract): ").lower()

    # Input state
    state = input("Enter state FIPS code or * for all (e.g., 18 for Indiana): ") or "*"

    # Input county if needed
    county = "*"
    if level in ['county', 'tract']:
        county = input("Enter county FIPS code or * for all: ") or "*"

    # Input tract if needed
    tract = "*"
    if level == 'tract':
        tract = input("Enter tract code or * for all: ") or "*"

    # Input API key if any
    key = input("Enter your Census API key (optional, press enter if none): ")

    # Build base URL
    base_url = f"https://api.census.gov/data/{year}/acs/{acs}/profile"

    # Build for and in parameters
    for_str = ""
    in_str = ""

    if level == 'state':
        for_str = f"state:{state}"
    elif level == 'county':
        for_str = f"county:{county}"
        if state != '*':
            in_str = f"state:{state}"
    elif level == 'tract':
        for_str = f"tract:{tract}"
        in_str = f"state:{state}"
        if county != '*':
            in_str += f" county:{county}"
        else:
            in_str += " county:*"
    else:
        print("Invalid level. Exiting.")
        return

    # Build full URL with descriptive=true
    url = f"{base_url}?get={get_params}&for={for_str}&descriptive=true"
    if in_str:
        url += f"&in={in_str}"
    if key:
        url += f"&key={key}"

    print(f"\nConstructed URL: {url}")

    # Fetch data
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if not data or len(data) < 2:
            print("No data returned.")
            return

        # Convert to DataFrame
        headers = data[0]
        rows = data[1:]
        df = pd.DataFrame(rows, columns=headers)

        # Save to CSV
        csv_filename = f"census_data_{year}_{acs}_{level}.csv"
        df.to_csv(csv_filename, index=False)

        # Download in Colab
        files.download(csv_filename)

        print(f"\nData downloaded successfully as {csv_filename}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except json.JSONDecodeError:
        print("Invalid JSON response. The API might not support the requested parameters.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# Run the function
download_census_data()